In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import math

In [11]:
B=3
C=4
H=5

X = torch.randn((B,C))
y = torch.randint(0,2,(B,))

w1 = torch.randn(C,H) / math.sqrt(C) / math.sqrt(2)
b1 = torch.randn(H)

w2 = torch.randn(H,H) / math.sqrt(C) / math.sqrt(2)
b2 = torch.randn(H)

w3 = torch.randn(H,2) / math.sqrt(H) / math.sqrt(2)
b3 = torch.randn(2)

params = [w1,b1,w2,b2,w3,b3]
for p in params:
    p.requires_grad=True

In [12]:
def check(a,b):
    assert a.shape == b.shape
    assert torch.isclose(a,b.grad).all(), f'{a}\n{b}'

In [13]:
logits.shape, act2.shape, w3.shape

(torch.Size([3, 2]), torch.Size([3, 5]), torch.Size([5, 2]))

In [21]:
lr = .1

for i in range(10):
    pre1 = X@w1+b1
    act1 = torch.relu(pre1)
    pre2 = act1@w2+b2
    act2 = torch.relu(pre2)
    logits = act2@w3+b3
    y_proba = torch.softmax(logits, dim=1)
    loss = -torch.log(y_proba[range(B), y]).mean()
    # break

    for p in [pre1,act1,pre2,act2,logits]:
        p.retain_grad()

    print(f'{loss:.2f}','acc', (y == y_proba.argmax(1)).sum() / B)

    for p in params:
        p.grad = None    
    loss.backward()
    # for p in params:
    #     p.data -= lr * p.grad
        
    dlogits = y_proba.clone()
    dlogits[range(B), y] -= 1
    dlogits /= B
    check(dlogits, logits)
    
    dw3 = act2.T@dlogits
    check(dw3, w3)
    dact2 = dlogits@w3.T
    check(dact2, act2)    
    db3 = dlogits.sum(0)
    check(db3, b3)
    dpre2 = torch.where(pre2>0, dact2, 0)
    check(dpre2, pre2)
    db2 = dpre2.sum(0)
    check(db2, b2)
    dw2 = act1.T@dpre2
    check(dw2, w2)    
    dact1 = dpre2@w2.T
    check(dact1, act1)    
    dpre1 = torch.where(pre1>0, dact1, 0)
    check(dpre1, pre1)    
    dw1 = X.T@dpre1
    check(dw1, w1)    
    db1 = dpre1.sum(0)

    # break
    dparams = [dw1,db1,dw2,db2,dw3,db3]

    for p,dp in zip(params, dparams):
        p.data -= lr * dp


    


0.14 acc tensor(1.)
0.13 acc tensor(1.)
0.13 acc tensor(1.)
0.12 acc tensor(1.)
0.11 acc tensor(1.)
0.11 acc tensor(1.)
0.10 acc tensor(1.)
0.10 acc tensor(1.)
0.09 acc tensor(1.)
0.09 acc tensor(1.)


In [22]:
dpre2.shape

torch.Size([3, 5])

In [ ]:
dpre

In [152]:
for p in params:
    print(p.grad)

None
None
None
None


acc tensor(0.6000)


In [153]:
y_proba

tensor([[0.5000, 0.5000],
        [0.9889, 0.0111],
        [0.5000, 0.5000],
        [0.9821, 0.0179],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.9806, 0.0194],
        [0.9875, 0.0125]], grad_fn=<SoftmaxBackward0>)

In [154]:
y_proba.argmax(1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [155]:
y

tensor([1, 0, 1, 0, 1, 1, 1, 1, 0, 0])